<a href="https://colab.research.google.com/github/AlvingEj/Arvenses/blob/main/Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import os

# Subir el archivo zip
uploaded = files.upload()

# Asegurarse de que el archivo zip esté presente en la subida
if uploaded:
    # Obtener el nombre del archivo zip subido
    zip_filename = list(uploaded.keys())[0]

    # Descomprimir el archivo zip
    try:
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall('dataset')
        print("Archivo zip descomprimido correctamente")
    except zipfile.BadZipFile:
        print("Error: El archivo subido no es un archivo zip válido.")
else:
    print("Error: No se subió ningún archivo zip.")


Saving dataset.zip to dataset (1).zip
Archivo zip descomprimido correctamente


In [ ]:
# Verificar la estructura de directorios
print("Contenido de la carpeta 'dataset':")
print(os.listdir('dataset'))

print("Contenido de la carpeta 'dataset/dataset':")
print(os.listdir('dataset/dataset'))

print("Contenido de la carpeta 'dataset/dataset/Manzana':")
print(os.listdir('dataset/dataset/Manzana'))
print("Contenido de la carpeta 'dataset/dataset/Uva':")
print(os.listdir('dataset/dataset/Uva'))


Contenido de la carpeta 'dataset':
['dataset']
Contenido de la carpeta 'dataset/dataset':
['Uva', 'Manzana']
Contenido de la carpeta 'dataset/dataset/Manzana':
['Manzana_Pondedumbre_Negra', 'Manzana_Sana', 'Manzana_Roya_Manzano', 'Manzana_Sarna_Manzano']
Contenido de la carpeta 'dataset/dataset/Uva':
['Uva_Sana', 'Uva_Sarampion_Negro', 'Uva_Podredumbre_Negra', 'Uva_Tizon_Hoja']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Generadores de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generadores de datos para la clasificación del tipo de planta
train_generator_type = datagen.flow_from_directory(
    'dataset/dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator_type = datagen.flow_from_directory(
    'dataset/dataset',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Generadores de datos para estado de las plantas para Manzana
train_generator_status_A = datagen.flow_from_directory(
    'dataset/dataset/Manzana',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_status_A = datagen.flow_from_directory(
    'dataset/dataset/Manzana',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Generadores de datos para estado de las plantas para Uva
train_generator_status_B = datagen.flow_from_directory(
    'dataset/dataset/Uva',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_status_B = datagen.flow_from_directory(
    'dataset/dataset/Uva',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



Found 5702 images belonging to 2 classes.
Found 1425 images belonging to 2 classes.
Found 2498 images belonging to 4 classes.
Found 622 images belonging to 4 classes.
Found 3208 images belonging to 4 classes.
Found 799 images belonging to 4 classes.


In [ ]:
# Modelo de Clasificación de Tipo de Planta
model_type = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model_type.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

# Resumen del modelo
model_type.summary()

# Entrenar el modelo
history_type = model_type.fit(
    train_generator_type,
    epochs=25,
    validation_data=validation_generator_type
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - accuracy: 0.7998 - loss: 0.4668 - val_accuracy: 0.9439 - val_loss: 0.1345
Epoch 2/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9632 - loss: 0.1058 - val_accuracy: 0.9593 - val_loss: 0.1175
Epoch 3/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.9806 - loss: 0.0588 - val_accuracy: 0.9172 - val_loss: 0.1971
Epoch 4/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.9910 - loss: 0.0305 - val_accuracy: 0.9733 - val_loss: 0.0716
Epoch 5/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9900 - loss: 0.0274 - val_accuracy: 0.9488 - val_loss: 0.1391
Epoch 6/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9804 - loss: 0.0587 - val_accuracy: 0.9312 - val_loss: 0.1898
Epoch 7/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - accuracy: 0.9874 - loss: 0.0329 - val_accuracy: 0.9439 - val_loss: 0.1509
Epoch 8/25
179/179 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.9975 - loss: 0.0088 - val_accuracy: 0

In [ ]:
# Modelo de Clasificación de Estado de la Planta para Manzana
model_status_A = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model_status_A.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

model_status_A.summary()

history_status_A = model_status_A.fit(
    train_generator_status_A,
    epochs=25,
    validation_data=validation_generator_status_A
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,035,716 (72.62 MB)

 Trainable params: 19,035,716 (72.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.5500 - loss: 1.2766 - val_accuracy: 0.8457 - val_loss: 0.4143
Epoch 2/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.8475 - loss: 0.4025 - val_accuracy: 0.8408 - val_loss: 0.4147
Epoch 3/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.8823 - loss: 0.3228 - val_accuracy: 0.9019 - val_loss: 0.2736
Epoch 4/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.9178 - loss: 0.2133 - val_accuracy: 0.9035 - val_loss: 0.2417
Epoch 5/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - accuracy: 0.9348 - loss: 0.1908 - val_accuracy: 0.9453 - val_loss: 0.1803
Epoch 6/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.9546 - loss: 0.1400 - val_accuracy: 0.9341 - val_loss: 0.1723
Epoch 7/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.9658 - loss: 0.1037 - val_accuracy: 0.9486 - val_loss: 0.1576
Epoch 8/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.9747 - loss: 0.0722 - val_accuracy: 0.9453 - 

In [ ]:
# Modelo de Clasificación de Estado de la Planta para Uva
model_status_B = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model_status_B.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

model_status_B.summary()

history_status_B = model_status_B.fit(
    train_generator_status_B,
    epochs=25,
    validation_data=validation_generator_status_B
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,035,716 (72.62 MB)

 Trainable params: 19,035,716 (72.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step - accuracy: 0.4811 - loss: 1.1999 - val_accuracy: 0.7772 - val_loss: 0.5902
Epoch 2/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.8234 - loss: 0.4679 - val_accuracy: 0.9287 - val_loss: 0.2223
Epoch 3/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - accuracy: 0.9110 - loss: 0.2546 - val_accuracy: 0.9399 - val_loss: 0.1644
Epoch 4/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.9443 - loss: 0.1697 - val_accuracy: 0.8974 - val_loss: 0.3422
Epoch 5/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9481 - loss: 0.1351 - val_accuracy: 0.9399 - val_loss: 0.1590
Epoch 6/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.9598 - loss: 0.1101 - val_accuracy: 0.9549 - val_loss: 0.1344
Epoch 7/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 12s 70ms/step - accuracy: 0.9774 - loss: 0.0685 - val_accuracy: 0.9136 - val_loss: 0.2174
Epoch 8/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.9667 - loss: 0.0895 - val_

In [ ]:
from google.colab import files
import tensorflow as tf
import numpy as np

# Define los mapeos de índices a etiquetas para uva y manzana
estado_mapeo_uv = {
    0: 'Uva Podredumbre Negra',
    1: 'Uva Sana',
    2: 'Uva Sarampión Negro',
    3: 'Uva Tizón Hoja'
}

estado_mapeo_ma = {
    0: 'Manzana Podredumbre Negra',
    1: 'Manzana Roya Manzano',
    2: 'Manzana Sana',
    3: 'Manzana Sarna Manzano'
}

# Subir archivos
uploaded = files.upload()

# Iterar sobre todas las imágenes cargadas
for filename in uploaded.keys():
    img_path = filename

    # Preprocesar la imagen
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0) / 255.0

    # Predecir el tipo de planta
    plant_type = model_type.predict(img_array)
    plant_type_label = 1 if plant_type[0][0] >= 0.5 else 0

    # Predecir el estado de la planta basado en el tipo
    if plant_type_label == 0:
        status = model_status_A.predict(img_array)
        estado_mapeo = estado_mapeo_ma
    else:
        status = model_status_B.predict(img_array)
        estado_mapeo = estado_mapeo_uv

    # Obtener la etiqueta del estado de la planta
    status_label_index = tf.argmax(status, axis=1).numpy()[0]
    status_label = estado_mapeo.get(status_label_index, 'Desconocido')

    # Imprimir resultados
    plant_type_name = "Manzana" if plant_type_label == 0 else "Uva"
    print(f"{filename}: Tipo de planta - {plant_type_name}, Estado de la planta - {status_label}")
